# Draft

In [1]:
import numpy as np
import scipy as sc
from scipy import signal

In [35]:
np.random.seed(24)

In [82]:
_filter = np.array([
    [0, 1, 0],
    [1, 1, 1],
    [0, 1, 0],
])

image = np.array([
    [0, 1, 0, 1, 0, 1, 0],
    [1, 1, 1, 1, 1, 1, 1],
    [0, 1, 0, 1, 0, 1, 0],
    [1, 1, 1, 1, 1, 1, 1],
    [0, 1, 0, 1, 0, 1, 0],
])

cnv_shape = (image.shape[0] - _filter.shape[0] + 1, image.shape[1] - _filter.shape[1] + 1)
cnv = np.zeros(cnv_shape)
cnv

for i in range(cnv_shape[0]):
    for j in range(cnv_shape[1]):
        cnv[i][j] = (image[i:i+_filter.shape[0], j:j+_filter.shape[0]] * _filter).sum()

cnv

array([[5., 3., 5., 3., 5.],
       [3., 4., 3., 4., 3.],
       [5., 3., 5., 3., 5.]])

In [3]:
image = np.ones((5, 7))
padd = 1
padding_image = np.zeros((image.shape[0]+padd*2, image.shape[1]+padd*2))
padding_image[padd:-padd, padd:-padd] = image
padding_image

array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [42]:
m = np.arange(16)
np.random.shuffle(m)
m = m.reshape(4, 4)

pooling = (2, 2)

In [43]:
m

array([[ 4,  3,  0,  2],
       [ 5,  8, 11, 10],
       [ 9, 15,  6,  7],
       [13,  1, 12, 14]])

In [51]:
m.shape

(4, 4)

In [62]:
result = np.zeros((int(m.shape[0]/pooling[0]), int(m.shape[1]/pooling[1])))
result = []
for i in range(0, m.shape[0]-pooling[0]+1, pooling[0]):
    for j in range(0, m.shape[1]-pooling[1]+1, pooling[1]):
        result.append(m[i:i+2, j:j+2].max())

result = np.array(result).reshape(int(m.shape[0]/pooling[0]), int(m.shape[1]/pooling[1]))
result

array([[ 8, 11],
       [15, 14]])

In [89]:
def get_cnv_zeros(image_shape: tuple, filter_shape: tuple) -> np.ndarray:
    cnv_shape = (image_shape[0] - filter_shape[0] + 1, image_shape[1] - filter_shape[1] + 1)
    return np.zeros(cnv_shape)


def get_padding(image: np.ndarray, padding: int=0) -> np.ndarray:
    padding_image = np.zeros((image.shape[0]+padding*2, image.shape[1]+padding*2))
    padding_image[padding:-padding, padding:-padding] = image
    return padding_image
    

def convolve(image: np.ndarray, _filter: np.ndarray, s_x: int=1, x_y: int=1, padding: int=0) -> np.ndarray:
    if padding:
        image = get_padding(image, padding)
        
    cnv = get_cnv_zeros(image.shape, _filter.shape)
    for i in range(cnv_shape[0]):
        for j in range(cnv_shape[1]):
            cnv[i][j] = (image[i:i+_filter.shape[0], j:j+_filter.shape[0]] * _filter).sum()
    return cnv


def pooling(m: np.ndarray, pooling: tuple, type_of_pooling='max-pooling') -> np.ndarray:
    if m.shape[0] % pooling[0] != 0 or m.shape[1] % pooling[1] != 0:
        return 'it\'s not okay'
    
    result = []
    for i in range(0, m.shape[0]-pooling[0]+1, pooling[0]):
        for j in range(0, m.shape[1]-pooling[1]+1, pooling[1]):
            result.append(m[i:i+pooling[0], j:j+pooling[1]].max())

    result = np.array(result).reshape(int(m.shape[0]/pooling[0]), int(m.shape[1]/pooling[1]))
    return result
    


In [5]:
get_padding(np.ones((3, 8)), 3)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [94]:
convolve(image, _filter, padding=2)

array([[0., 0., 1., 0., 1., 0., 0., 0., 0.],
       [0., 2., 2., 3., 2., 0., 0., 0., 0.],
       [1., 2., 5., 3., 5., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [96]:
c = convolve(image, _filter, padding=2)
pooling(c, (1, 3))

array([[1., 1., 0.],
       [2., 3., 0.],
       [5., 5., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [97]:
get_cnv_zeros(image.shape, _filter.shape)

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [8]:
_filter = np.array(
    [[0, 1, 0],
     [1, 1, 1],
     [0, 1, 0]
    ]
)

image = np.array([
    [0, 1, 0, 1, 0, 1, 0],
    [1, 1, 1, 1, 1, 1, 1],
    [0, 1, 0, 1, 0, 1, 0],
    [1, 1, 1, 1, 1, 1, 1],
    [0, 1, 0, 1, 0, 1, 0]
    ])

s_x=1
s_y=1

res = signal.convolve2d(image, _filter[::-1, ::-1], mode='valid')[::s_y, ::s_x]
res

array([[5, 3, 5, 3, 5],
       [3, 4, 3, 4, 3],
       [5, 3, 5, 3, 5]])